# Transformação para formato Time Series

## Pacotes necessários

In [14]:
import pandas as pd
import fastparquet
import os

## Leitura dos dados

In [ ]:
# Carregar o dataframe recodificado

# Costela
#df = pd.read_parquet("/home/costela/Documentos/Escorpiao-series/Dados_processados/df1_recodificado.parquet")

# Usuario
df = pd.read_parquet("/home/usuario/Documentos/Escorpiao_series/Dados_processados/df1_recodificado.parquet")

# Filtrar os dados para incluir apenas os anos de 2014 a 2023
df = df[(df['ANO'] >= 2014) & (df['ANO'] <= 2023)]


## Agrupamento e calculo da Tx Incidencia

In [ ]:
# Agrupar os dados por município, ano, mês e população
dfg = (
    df
    .groupby(['NOME_MUNI', 'ANO', 'MES', 'POPULACAO'])
    .size()
    .reset_index(name='CASOS')
)

## Criar um dataframe por municipio

In [28]:
# -------------------------------------------------------
# Criar coluna DATA (primeiro dia de cada mês)
# -------------------------------------------------------
dfg['DATA'] = pd.to_datetime(
    dfg['ANO'].astype(str) + '-' + dfg['MES'].astype(str) + '-01'
)

# -------------------------------------------------------
# Completar todos os meses para cada município
# -------------------------------------------------------
lista_municipios = []

for municipio, df_mun in dfg.groupby('NOME_MUNI'):

    # Garantir ordenação
    df_mun = df_mun.sort_values('DATA')

    # Criar sequência mensal completa
    datas_completas = pd.date_range(
        start=df_mun['DATA'].min(),
        end=df_mun['DATA'].max(),
        freq='MS'
    )

    # Reindexar para forçar todos os meses
    df_mun = (
        df_mun
        .set_index('DATA')
        .reindex(datas_completas)
        .rename_axis('DATA')
        .reset_index()
    )

    # Preencher nome do município
    df_mun['NOME_MUNI'] = municipio

    # Recriar ANO e MES
    df_mun['ANO'] = df_mun['DATA'].dt.year
    df_mun['MES'] = df_mun['DATA'].dt.month

    # Preencher meses sem casos
    df_mun['CASOS'] = df_mun['CASOS'].fillna(0)

    # Preencher população (constante por município)
    df_mun['POPULACAO'] = df_mun['POPULACAO'].ffill().bfill()

    lista_municipios.append(df_mun)


In [29]:
# Unir todos os municípios em um único DataFrame
df_mensal = pd.concat(lista_municipios, ignore_index=True)

# Calcular taxa mensal por 100.000 habitantes
df_mensal['TAXA_MENSAL'] = (
    df_mensal['CASOS'] / df_mensal['POPULACAO']
) * 100000

df_mensal['TAXA_MENSAL'] = df_mensal['TAXA_MENSAL'].round(1)

## Salvar Dataframe

In [31]:
df_mensal = df_mensal.sort_values(['NOME_MUNI', 'DATA']).reset_index(drop=True)

df_mensal.to_parquet(
    '/home/usuario/Documentos/Escorpiao_series/Dados_processados/series_mensal_model_ready.parquet',
    index=False
)                            